In this example we are going to scrape board actions from the [Texas Medical Board](https://profile.tmb.state.tx.us/SearchBA.aspx).

Traditionally Python programmers use [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/) to scrape content from the interent. Instead of being *traditional*, we're going to use [Playwright](https://playwright.dev/python/), a **browser automation tool**! This means you actually control the browser! Filling out forms, clicking buttons, downloading documents... it's magic!!!✨✨✨

# Texas Medical Board Actions

- Filling out text inputs
- Inspecting the page
- Looping through date ranges
- Combining dataframes

## Installation

We need to install a few tools first! Remove the `#` and run the cell to install the Python packages and browsers that we'll need for our scraping adventure.

In [ ]:
# %pip install --quiet lxml html5lib beautifulsoup4 pandas
# %pip install --quiet playwright
# !playwright install

## Opening up the browser and visiting our destination


In [ ]:
from playwright.async_api import async_playwright

# "Hey, open up a browser"
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)

# Create a new browser window
page = await browser.new_page()

In [ ]:
await page.goto("https://profile.tmb.state.tx.us/SearchBA.aspx")

## Filling in a text field

You always start with `await page.locator("input").fill("whatever you want")`. You'll probably get an error because there are multiple inputs on the page, but Playwright doesn't know which one you want to use! Just read the error and figure out the right one.

In [ ]:
#await page.locator("input").fill("3/11/2021")
await page.locator("#BodyContent_tbBADate").fill("3/11/2021")
await page.locator("#BodyContent_tbBADateRangeEnd").fill("3/1/2024")

## Click the search button and wait for the results to show up

In [ ]:
await page.get_by_role("button", name="Search").click()

In [ ]:
await page.get_by_text("Board Actions").wait_for()

## Grab the tables from the page

[Pandas](https://pandas.pydata.org/) is the Python equivalent to Excel, and it's great at dealing with tabular data! Often the data on a web page that looks like a spreadsheet can be read with `pd.read_html`.

You use `await page.content()` to save the contents of the page, then feed it to `read_html` to find the tables. `len(tables)` checks the number of tables you have, then you manually poke around to see which one is the one you're interested in. `tables[0]` is the first one, `tables[1]` is the second one, and so on...

In [ ]:
from io import StringIO
import pandas as pd

html = await page.content()
tables = pd.read_html(StringIO(html))
len(tables)

In [ ]:
tables[1]

In [ ]:
#await page.get_by_text("Back").click()
await page.get_by_role("link", name="Back").click()

## Cycling through dates

We'll start from one date, and work forward to another date. This is just to see what it looks like.

In [ ]:
import datetime

start_date = datetime.date(2023, 4, 1)
end_date = datetime.date(2023, 4, 10)

# Loop over each day from the start date to the end date
current_date = start_date
while current_date <= end_date:
    current_date = current_date + datetime.timedelta(days=1)
    date_str = current_date.strftime("%m/%d/%Y")
    print("Searching", date_str)

## Putting it all together

Now we'll visit the page, go through each date, saving the results to our `all_data` overall list of data.

In [ ]:
await page.goto("https://profile.tmb.state.tx.us/SearchBA.aspx?eb2d4a70-6591-4ad4-ae6d-a2727e84cb39")

In [ ]:
import datetime
import pandas as pd
from io import StringIO

all_data = pd.DataFrame()

start_date = datetime.date(2023, 4, 1)
end_date = datetime.date(2023, 5, 1)

# Loop over each day from the start date to the end date
current_date = start_date
while current_date <= end_date:
    current_date = current_date + datetime.timedelta(days=1)
    date_str = current_date.strftime("%m/%d/%Y")
    
    print("Searching", date_str)
    
    await page.locator("#BodyContent_tbBADate").fill(date_str)
    await page.locator("#BodyContent_tbBADateRangeEnd").fill(date_str)

    await page.get_by_role("button", name="Search").click()

    try:
        await page.get_by_text("Board Actions").wait_for()
        html = await page.content()
        tables = pd.read_html(StringIO(html))
        df = tables[1]
    
        all_data = pd.concat([all_data, df], ignore_index = True)
    except:
        print("Failed")

    await page.get_by_role("link", name="Back").click()

In [ ]:
all_data

## Saving the results

Now we'll save it to a CSV file! Easy peasy.

In [ ]:
all_data.to_csv("licenses.csv", index=False)